In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import datetime
import seaborn as sns

In [4]:
pay_data = pd.read_csv('PAYMENTS.CSV')
Active_data = pd.read_csv('ACTIVE_USER.CSV')
Demographics_data = pd.read_csv('DEMOGRAFICOS.CSV')
Account_data = pd.read_csv('DINERO_CUENTA.CSV')
Evaluate_data = pd.read_csv('EVALUATE.CSV')
MarketPlace_data = pd.read_csv('MARKETPLACE_data.CSV')

# Demographics_data

In [5]:
#ok ideas.
#city tiene 0.05 de nulls. fillna y a dormir
#no diferencia caba de capital federal. entiendo que es lo mismo asi que junto
#tiene buenos aires y diferencia la plata y otros barrios. fijate eso bien
#palermo aparece separado de caba. dejalo separado. por ahi en palermo se hacen mas compras que en el resto de barrios individualmente y por eso esta separado
#esta buenos aires y los municipiso separados. por ahi los de buenos aires no encontraron municipio o no importaba
#el estado esta bn. algunos muy pocos desp vuelan
#el genero hay 2k de unknown. fijate si compran mas o mneos que el promedio. si es lo mismo metelos en algun genero y q no rompan
#tarjeta esta bien
#en edad tengo que hacer un regex.dejalo en bin y es categorica 
#hay 2k sin dato.fill con mode?

In [6]:
Demographics_data.head()

,CITY,CUS_CUST_ID_BUY,GENDER,RANGO_EDAD,TARJETAS,ESTADO
0,caba,47629682,male,03.Entre 26 y 30 años,Credit Card,capital federal
1,belen de escobar,184288617,male,03.Entre 26 y 30 años,Account Money,buenos aires
2,san fernando del valle de catamarca,134359455,female,03.Entre 26 y 30 años,Debit Card,catamarca
3,lomas de zamora,126256149,female,07.Entre 56 y 65 años,Account Money,buenos aires
4,villa manzano,447582139,male,02.Entre 18 y 25 años,Debit Card,rio negro


In [7]:
Demographics_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30466 entries, 0 to 30465
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CITY             28711 non-null  object
 1   CUS_CUST_ID_BUY  30466 non-null  int64 
 2   GENDER           30466 non-null  object
 3   RANGO_EDAD       30466 non-null  object
 4   TARJETAS         30466 non-null  object
 5   ESTADO           30466 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.4+ MB


In [8]:
Demographics_data.isnull().sum()/Demographics_data.shape[0]

CITY               0.057605
CUS_CUST_ID_BUY    0.000000
GENDER             0.000000
RANGO_EDAD         0.000000
TARJETAS           0.000000
ESTADO             0.000000
dtype: float64

In [9]:
Demographics_data.CITY.value_counts()

caba                             1784
capital federal                  1440
buenos aires                      672
cordoba                           521
la plata                          481
                                 ... 
capital barrio la cruz              1
portezuelo                          1
barrio norte, capital federal       1
avellaneda dominico                 1
cachari                             1
Name: CITY, Length: 3155, dtype: int64

In [10]:
Demographics_data.CITY[(Demographics_data.CITY != 'caba') & (Demographics_data.CITY != 'capital federal') & 
                       (Demographics_data.CITY != 'buenos aires') & (Demographics_data.CITY != 'la plata')
                      & (Demographics_data.CITY != 'cordoba')].value_counts()

rosario                              416
palermo                              375
mar del plata                        368
lomas de zamora                      265
moreno                               257
                                    ... 
fatima                                 1
villa parque caseros 3 de febrero      1
beraxatrhui                            1
villa elisa barrio el rincon           1
villa ocanpo varrio norte              1
Name: CITY, Length: 3150, dtype: int64

In [11]:
patern_caba = r'\.*capital federal\.*'
patern_caba_regex = re.compile(patern_caba,re.IGNORECASE)
caba_Column = Demographics_data.CITY.apply(lambda x: x if x is np.NaN else patern_caba_regex.sub('caba',x))
Demographics_data["caba_Ok"] = caba_Column

In [12]:
Demographics_data.caba_Ok.value_counts()

caba                             3224
buenos aires                      672
cordoba                           521
la plata                          481
rosario                           416
                                 ... 
corrientesarazaty                   1
comandante luis jorge fontana       1
racedo                              1
alem y cordoba                      1
villa ocanpo varrio norte           1
Name: caba_Ok, Length: 3147, dtype: int64

In [13]:
1784 + 1440

3224

In [14]:
Demographics_data.caba_Ok.fillna('buenos aires',inplace = True)

In [15]:
Demographics_data.ESTADO.value_counts()

buenos aires            13367
capital federal          8455
cordoba                  1356
santa fe                 1202
mendoza                   622
entre rios                579
tucuman                   434
rio negro                 392
neuquen                   391
misiones                  391
salta                     378
corrientes                367
chaco                     363
san luis                  283
chubut                    279
san juan                  267
santa cruz                242
formosa                   216
jujuy                     209
la rioja                  146
gran buenos aires         132
catamarca                 126
santiago del estero       124
la pampa                  108
tierra del fuego           25
virginia                    6
alto parana                 1
são paulo                   1
minnesota                   1
bs.as. g.b.a. norte         1
buenos aires interio        1
nuevo leon                  1
Name: ESTADO, dtype: int64

In [16]:
Demographics_data.GENDER.value_counts()/Demographics_data.shape[0]

male       0.522353
female     0.419123
Unknown    0.058524
Name: GENDER, dtype: float64

In [17]:
Demographics_data.GENDER.value_counts()

male       15914
female     12769
Unknown     1783
Name: GENDER, dtype: int64

In [18]:
Demographics_data.TARJETAS.value_counts()

Credit Card         14308
Account Money        7717
Debit Card           7440
Ticket                945
Bank Transfer          20
digital_currency       19
ATM                    17
Name: TARJETAS, dtype: int64

In [19]:
Demographics_data.RANGO_EDAD.value_counts()

06.Entre 41 y 55 años    6822
04.Entre 31 y 35 años    5155
03.Entre 26 y 30 años    5072
05.Entre 36 y 40 años    4331
02.Entre 18 y 25 años    4246
00. Sin Dato             2700
07.Entre 56 y 65 años    1545
08.Mayor de 65 años       541
01.Menor de 18 años        54
Name: RANGO_EDAD, dtype: int64

In [20]:
Demographics_data['edad_sin_null'] = Demographics_data.RANGO_EDAD.replace (to_replace='00. Sin Dato', value='06.Entre 41 y 55 años')

In [21]:
Demographics_data['edad_sin_null'].value_counts()

06.Entre 41 y 55 años    9522
04.Entre 31 y 35 años    5155
03.Entre 26 y 30 años    5072
05.Entre 36 y 40 años    4331
02.Entre 18 y 25 años    4246
07.Entre 56 y 65 años    1545
08.Mayor de 65 años       541
01.Menor de 18 años        54
Name: edad_sin_null, dtype: int64

In [22]:
Demographics_data['edad_split'] = Demographics_data['edad_sin_null'].str.split()

In [23]:
Demographics_data['edad_split']

0        [03.Entre, 26, y, 30, años]
1        [03.Entre, 26, y, 30, años]
2        [03.Entre, 26, y, 30, años]
3        [07.Entre, 56, y, 65, años]
4        [02.Entre, 18, y, 25, años]
                    ...             
30461    [02.Entre, 18, y, 25, años]
30462    [06.Entre, 41, y, 55, años]
30463    [06.Entre, 41, y, 55, años]
30464    [03.Entre, 26, y, 30, años]
30465    [07.Entre, 56, y, 65, años]
Name: edad_split, Length: 30466, dtype: object

In [24]:
Demographics_data['Edad_ok'] = Demographics_data['edad_split'].apply(lambda x: '{}-{}'.format(x[1],x[3]))

In [25]:
Demographics_data.Edad_ok.value_counts()

41-55      9522
31-35      5155
26-30      5072
36-40      4331
18-25      4246
56-65      1545
de-años     595
Name: Edad_ok, dtype: int64

In [26]:
Demographics_data['Edad_ok'] = Demographics_data.Edad_ok.replace (to_replace='de-años', value='0-18')

In [27]:
Demographics_data.Edad_ok.value_counts()

41-55    9522
31-35    5155
26-30    5072
36-40    4331
18-25    4246
56-65    1545
0-18      595
Name: Edad_ok, dtype: int64

In [28]:
Demographics_data.drop(columns = ['CITY','RANGO_EDAD','edad_sin_null','edad_split'],inplace = True)

In [29]:
Demographics_data.rename(columns = {'caba_Ok':'CITY','Edad_ok': 'Rango_edad'},inplace = True)

In [30]:
Demographics_data.CUS_CUST_ID_BUY.value_counts()

186580992    1
204317409    1
139883259    1
295074554    1
63997360     1
            ..
314928358    1
75232852     1
332274996    1
1099346      1
142084095    1
Name: CUS_CUST_ID_BUY, Length: 30466, dtype: int64

In [31]:
Demographics_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30466 entries, 0 to 30465
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CUS_CUST_ID_BUY  30466 non-null  int64 
 1   GENDER           30466 non-null  object
 2   TARJETAS         30466 non-null  object
 3   ESTADO           30466 non-null  object
 4   CITY             30466 non-null  object
 5   Rango_edad       30466 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.4+ MB


# pay_data

In [32]:
#ideas

#contar cuantas compras hizo
#sacar el porcentuaje promedio de descuento que tuvo el tipo
#el total de lo que gasto
#4 columnas con la cantidad que hizo en trasnporte,celu,qr y servicios
#un par metieron 2.5k de transacciones dales el premio al consumidor del mes
#la fecha va del 1 al 7. 7 meses. por ahi poner transacciones de cada mes en ves de todo junto (o por ahi trimestre para que no sea tanto)
#pongamos 4 columnas con si o no si le compro a los 4 id que mas venden

#mes 6 y 5 fiestas? estaria bueno poner como una feature dias de descuento o dias que la gente haga regalos y poner cuantos dias antes
#y despues hacen en promedio por ahi. tipo este flaco suele comprar en cualquier momento o este tipo solo compra en epocas cercanas
#a regalar cosas o solo cuando hay descuentos. por ahi decis churn 1 pero en realidad el tipo va a volver en fiestas o en descuentos

In [33]:
pay_data.head(2)

,FECHA,CUS_CUST_ID_SEL,CUS_CUST_ID_BUY,SPENT,TPV_SEGMENT_DETAIL,DESCUENTO
0,2019-05-25,314941456,20663512,11.3566,Instore,2.2312
1,2019-06-20,251693291,444286946,4.6190,Transport,0.9238


In [34]:
pay_data.CUS_CUST_ID_BUY.value_counts()

313138772    2700
211115419    2477
237755472    1579
356174727    1166
201543339    1138
             ... 
17199580        1
17361423        1
184164950       1
436890207       1
443549695       1
Name: CUS_CUST_ID_BUY, Length: 43502, dtype: int64

In [35]:
pay_data.FECHA.min()

'2019-01-01'

In [36]:
pay_data.FECHA.max()

'2019-07-01'

In [37]:
pay_data.CUS_CUST_ID_SEL.value_counts()

251693291    141306
254303242    115114
295153733     58086
226295793     57262
362615356     16117
              ...  
423556858         1
256796392         1
89493697          1
267120217         1
257637991         1
Name: CUS_CUST_ID_SEL, Length: 37389, dtype: int64

In [38]:
pay_data.TPV_SEGMENT_DETAIL.value_counts() 

Cellphone Recharge      173199
Transport               141306
Instore                 138801
Utilities                57262
Money Transfer           44946
Other Single Players         2
Name: TPV_SEGMENT_DETAIL, dtype: int64

In [39]:
pay_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555516 entries, 0 to 555515
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   FECHA               555516 non-null  object 
 1   CUS_CUST_ID_SEL     555516 non-null  int64  
 2   CUS_CUST_ID_BUY     555516 non-null  int64  
 3   SPENT               555516 non-null  float64
 4   TPV_SEGMENT_DETAIL  555516 non-null  object 
 5   DESCUENTO           555516 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 25.4+ MB


In [40]:
pay_data['month'] = pd.DatetimeIndex(pay_data['FECHA']).month

In [41]:
pay_data.month.value_counts() 

6    133231
5    114521
4     93275
3     81098
2     65086
1     62406
7      5899
Name: month, dtype: int64

In [42]:
list_uniques = pay_data.CUS_CUST_ID_BUY.unique()

In [43]:
list_months = [1,2,3,4,5,6,7]
def get_months_ammount(ID):
    r = pay_data.month[pay_data.CUS_CUST_ID_BUY == ID].value_counts()    
    z = pd.DataFrame(r).transpose()
    
    for x in list_months:
        if x not in z.columns:
            z[x] = 0
    z =  z.assign(CUS_CUST_ID_BUY = ID).reset_index(drop=True)
    return z

In [44]:
dataframe_months = pd.DataFrame()

In [45]:
for x in list_uniques:
    data = get_months_ammount(x)
    dataframe_months = dataframe_months.append(data)    

In [46]:
for x in pay_data.CUS_CUST_ID_BUY.unique():
    if x not in dataframe_months.CUS_CUST_ID_BUY.unique():
        print(x)

In [47]:
pay_data = pay_data.merge(dataframe_months, on = 'CUS_CUST_ID_BUY' )

In [48]:
pay_data = pay_data.drop(columns = ['FECHA','month']).rename(columns = {1:'1_trans',2:'2_trans',3:'3_trans',4:'4_trans',5:'5_trans',6:'6_trans',7:'7_trans'})

In [49]:
pay_data['total_transactions'] = pay_data[['1_trans','3_trans','4_trans','6_trans','2_trans','5_trans','7_trans']].sum(axis = 1)

In [50]:
total_spent = pay_data[['CUS_CUST_ID_BUY','SPENT']].groupby('CUS_CUST_ID_BUY').sum()

In [51]:
total_spent['CUS_CUST_ID_BUY'] = total_spent.index

In [52]:
total_spent = total_spent.reset_index(drop = True)

In [53]:
pay_data = pay_data.merge(total_spent, on = 'CUS_CUST_ID_BUY').rename(columns = {'SPENT_y':'TotalSpent'})

In [54]:
r = pay_data[['DESCUENTO','SPENT_x','CUS_CUST_ID_BUY']]

In [55]:
r['desc_porc'] = (r['DESCUENTO']  * 100 )/(r['DESCUENTO'] + r['SPENT_x'])

C:\Users\gonzalo\.conda\envs\personales\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
r.drop(columns =['DESCUENTO','SPENT_x'],inplace = True)

C:\Users\gonzalo\.conda\envs\personales\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [57]:
discount_group = r.groupby('CUS_CUST_ID_BUY').mean()

In [58]:
discount_group['CUS_CUST_ID_BUY'] = discount_group.index

In [59]:
discount_group = discount_group.reset_index(drop = True)

In [60]:
pay_data = pay_data.merge(discount_group, on = 'CUS_CUST_ID_BUY').rename(columns = {'desc_porc':'Discount_Mean'})

In [61]:
id_sell_data = pay_data[(pay_data.CUS_CUST_ID_SEL == 251693291)|(pay_data.CUS_CUST_ID_SEL == 254303242)|(pay_data.CUS_CUST_ID_SEL == 295153733)|(pay_data.CUS_CUST_ID_SEL == 226295793)]

In [62]:
id_sell_data

,CUS_CUST_ID_SEL,CUS_CUST_ID_BUY,SPENT_x,TPV_SEGMENT_DETAIL,DESCUENTO,1_trans,3_trans,4_trans,6_trans,2_trans,5_trans,7_trans,total_transactions,TotalSpent,Discount_Mean
1,251693291,20663512,5.3519,Transport,1.0704,4,2,2,2,1,1,0,12,193.6114,8.312826
2,251693291,20663512,4.5589,Transport,0.0000,4,2,2,2,1,1,0,12,193.6114,8.312826
3,251693291,20663512,8.0278,Transport,0.0000,4,2,2,2,1,1,0,12,193.6114,8.312826
4,251693291,20663512,5.0000,Transport,1.0000,4,2,2,2,1,1,0,12,193.6114,8.312826
9,251693291,20663512,4.7416,Transport,0.9483,4,2,2,2,1,1,0,12,193.6114,8.312826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555504,251693291,157593930,2.5465,Transport,0.5093,0,0,0,0,1,0,0,1,2.5465,16.666667
555505,254303242,191571237,3.3467,Cellphone Recharge,0.0000,0,0,0,0,0,1,0,1,3.3467,0.000000
555507,254303242,239848548,1.1136,Cellphone Recharge,0.5568,0,0,0,1,0,0,0,1,1.1136,33.333333
555508,254303242,292867782,4.4356,Cellphone Recharge,0.0000,0,0,0,0,0,1,0,1,4.4356,0.000000


In [63]:
id_sell_data['sell_id_251693291'] = id_sell_data.CUS_CUST_ID_SEL.apply(lambda x: 1 if x == 251693291 else 0)

C:\Users\gonzalo\.conda\envs\personales\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
id_sell_data['sell_id_254303242'] = id_sell_data.CUS_CUST_ID_SEL.apply(lambda x: 1 if x == 254303242 else 0)

C:\Users\gonzalo\.conda\envs\personales\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
id_sell_data['sell_id_295153733'] = id_sell_data.CUS_CUST_ID_SEL.apply(lambda x: 1 if x == 295153733 else 0)

C:\Users\gonzalo\.conda\envs\personales\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [66]:
id_sell_data['sell_id_226295793'] = id_sell_data.CUS_CUST_ID_SEL.apply(lambda x: 1 if x == 226295793 else 0)

C:\Users\gonzalo\.conda\envs\personales\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [67]:
id_sell_data = id_sell_data[['CUS_CUST_ID_BUY','sell_id_251693291','sell_id_254303242','sell_id_295153733','sell_id_226295793']]

In [68]:
id_sell_data = id_sell_data.groupby('CUS_CUST_ID_BUY', group_keys=False).apply(lambda id_sell_data: id_sell_data.sample(1))

In [69]:
pay_data = pay_data.merge(id_sell_data, on = 'CUS_CUST_ID_BUY',how = 'outer').fillna(0)

In [70]:
pay_data.TPV_SEGMENT_DETAIL.value_counts()

Cellphone Recharge      173199
Transport               141306
Instore                 138801
Utilities                57262
Money Transfer           44946
Other Single Players         2
Name: TPV_SEGMENT_DETAIL, dtype: int64

In [71]:
pay_data.TPV_SEGMENT_DETAIL[pay_data.TPV_SEGMENT_DETAIL == 'Other Single Players']

376327    Other Single Players
490670    Other Single Players
Name: TPV_SEGMENT_DETAIL, dtype: object

In [72]:
pay_data.at[376327,'TPV_SEGMENT_DETAIL'] = 'Cellphone Recharge'

In [73]:
pay_data.at[490670,'TPV_SEGMENT_DETAIL'] = 'Cellphone Recharge'

In [74]:
data_segment = pay_data[['CUS_CUST_ID_BUY','TPV_SEGMENT_DETAIL']]

In [75]:
data_segment.TPV_SEGMENT_DETAIL.value_counts()

Cellphone Recharge    173201
Transport             141306
Instore               138801
Utilities              57262
Money Transfer         44946
Name: TPV_SEGMENT_DETAIL, dtype: int64

In [76]:
data_segment['Cellphone Recharge'] = data_segment.TPV_SEGMENT_DETAIL.apply(lambda x: 1 if x == 'Cellphone Recharge' else 0)

C:\Users\gonzalo\.conda\envs\personales\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
data_segment['Transport'] = data_segment.TPV_SEGMENT_DETAIL.apply(lambda x: 1 if x == 'Transport' else 0)

C:\Users\gonzalo\.conda\envs\personales\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
data_segment['Instore'] = data_segment.TPV_SEGMENT_DETAIL.apply(lambda x: 1 if x == 'Instore' else 0)

C:\Users\gonzalo\.conda\envs\personales\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [79]:
data_segment['Utilities'] = data_segment.TPV_SEGMENT_DETAIL.apply(lambda x: 1 if x == 'Utilities' else 0)

C:\Users\gonzalo\.conda\envs\personales\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
data_segment['Money Transfer'] = data_segment.TPV_SEGMENT_DETAIL.apply(lambda x: 1 if x == 'Money Transfer' else 0)

C:\Users\gonzalo\.conda\envs\personales\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [81]:
data_segment = data_segment.groupby('CUS_CUST_ID_BUY', group_keys=False).apply(lambda data_segment: data_segment.sample(1))

In [82]:
pay_data = pay_data.merge(data_segment, on = 'CUS_CUST_ID_BUY')

In [83]:
pay_data.drop(columns = ['CUS_CUST_ID_SEL','SPENT_x','TPV_SEGMENT_DETAIL_x','DESCUENTO','TPV_SEGMENT_DETAIL_y'],inplace = True)

In [84]:
pay_data = pay_data.groupby('CUS_CUST_ID_BUY', group_keys=False).apply(lambda pay_data: pay_data.sample(1))

In [85]:
pay_data.head()

,CUS_CUST_ID_BUY,1_trans,3_trans,4_trans,6_trans,2_trans,5_trans,7_trans,total_transactions,TotalSpent,Discount_Mean,sell_id_251693291,sell_id_254303242,sell_id_295153733,sell_id_226295793,Cellphone Recharge,Transport,Instore,Utilities,Money Transfer
375635,1276,0,1,0,0,2,1,0,4,7.6365,0.000000,0.0,0.0,1.0,0.0,1,0,0,0,0
151477,14525,2,10,11,2,8,13,0,46,429.3148,4.449163,0.0,0.0,0.0,0.0,0,0,1,0,0
204812,21230,0,0,0,0,0,1,0,1,4.4198,33.333333,1.0,0.0,0.0,0.0,0,1,0,0,0
24842,22083,3,5,1,1,4,4,0,18,196.6036,8.628314,1.0,0.0,0.0,0.0,0,0,1,0,0
552323,23513,0,3,4,0,2,1,0,10,31.1145,0.000000,1.0,0.0,0.0,0.0,0,1,0,0,0


In [86]:
pay_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43502 entries, 375635 to 295212
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CUS_CUST_ID_BUY     43502 non-null  int64  
 1   1_trans             43502 non-null  int64  
 2   3_trans             43502 non-null  int64  
 3   4_trans             43502 non-null  int64  
 4   6_trans             43502 non-null  int64  
 5   2_trans             43502 non-null  int64  
 6   5_trans             43502 non-null  int64  
 7   7_trans             43502 non-null  int64  
 8   total_transactions  43502 non-null  int64  
 9   TotalSpent          43502 non-null  float64
 10  Discount_Mean       43502 non-null  float64
 11  sell_id_251693291   43502 non-null  float64
 12  sell_id_254303242   43502 non-null  float64
 13  sell_id_295153733   43502 non-null  float64
 14  sell_id_226295793   43502 non-null  float64
 15  Cellphone Recharge  43502 non-null  int64  
 16

In [87]:
Final_Data = Demographics_data.merge(pay_data,how = 'outer',on = 'CUS_CUST_ID_BUY')

In [88]:
Final_Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43502 entries, 0 to 43501
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CUS_CUST_ID_BUY     43502 non-null  int64  
 1   GENDER              30466 non-null  object 
 2   TARJETAS            30466 non-null  object 
 3   ESTADO              30466 non-null  object 
 4   CITY                30466 non-null  object 
 5   Rango_edad          30466 non-null  object 
 6   1_trans             43502 non-null  int64  
 7   3_trans             43502 non-null  int64  
 8   4_trans             43502 non-null  int64  
 9   6_trans             43502 non-null  int64  
 10  2_trans             43502 non-null  int64  
 11  5_trans             43502 non-null  int64  
 12  7_trans             43502 non-null  int64  
 13  total_transactions  43502 non-null  int64  
 14  TotalSpent          43502 non-null  float64
 15  Discount_Mean       43502 non-null  float64
 16  sell

In [89]:
Final_Data.fillna('Unknown',inplace = True)

In [90]:
Final_Data.GENDER.value_counts()

male       15914
Unknown    14819
female     12769
Name: GENDER, dtype: int64

# ACTIVE_USER

    * CUS_CUST_ID_BUY: ID del buyer.
    * MAU_MP_1: Cantidad de veces el usuario entró a la APP  de MP (azul) en el último mes de entrenamiento (mes -1).
    * MAU_ML_1: Cantidad de veces el usuario entró a la APP  de ML (amarilla) en el último mes de entrenamiento (mes -1).
    * MAU_ML_2: Cantidad de veces el usuario entró a la APP  de ML (amarilla) en el mes -2 de entrenamiento.
    * last_login_mp_date_1: Fecha del último login del mes en APP MP.

In [91]:
#separar fecha en dia mes
#cant de logins totales en los ultimos 3 meses de mp y ml
#rellenar nulls

In [92]:
Active_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43502 entries, 0 to 43501
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CUS_CUST_ID_BUY       43502 non-null  int64  
 1   MAU_MP_3              33016 non-null  float64
 2   MAU_ML_3              33016 non-null  float64
 3   MAU_MP_2              35257 non-null  float64
 4   MAU_ML_2              35257 non-null  float64
 5   MAU_MP_1              37001 non-null  float64
 6   MAU_ML_1              37001 non-null  float64
 7   last_login_mp_date_1  37001 non-null  object 
 8   last_login_ml_date_1  37001 non-null  object 
dtypes: float64(6), int64(1), object(2)
memory usage: 3.0+ MB


In [93]:
Active_data.head()

,CUS_CUST_ID_BUY,MAU_MP_3,MAU_ML_3,MAU_MP_2,MAU_ML_2,MAU_MP_1,MAU_ML_1,last_login_mp_date_1,last_login_ml_date_1
0,424070401,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN
1,271894453,1.0,14.0,11.0,21.0,2.0,12.0,2019-05-17,2019-05-24
2,327402711,NaN,NaN,0.0,10.0,4.0,26.0,2019-05-28,2019-05-31
3,310355113,5.0,18.0,5.0,7.0,3.0,10.0,2019-05-21,2019-05-29
4,59350476,1.0,18.0,5.0,17.0,4.0,27.0,2019-05-31,2019-05-31


In [94]:
Active_data.CUS_CUST_ID_BUY.unique

<bound method Series.unique of 0        424070401
1        271894453
2        327402711
3        310355113
4         59350476
           ...    
43497     84534152
43498     59319045
43499     80894929
43500    237046723
43501     49771756
Name: CUS_CUST_ID_BUY, Length: 43502, dtype: int64>

In [95]:
Active_data.isnull().sum()/Active_data.shape[0]

CUS_CUST_ID_BUY         0.000000
MAU_MP_3                0.241046
MAU_ML_3                0.241046
MAU_MP_2                0.189532
MAU_ML_2                0.189532
MAU_MP_1                0.149441
MAU_ML_1                0.149441
last_login_mp_date_1    0.149441
last_login_ml_date_1    0.149441
dtype: float64

In [96]:
Active_data.MAU_MP_3.value_counts().head(3)

0.0    8703
1.0    4122
2.0    3153
Name: MAU_MP_3, dtype: int64

In [97]:
Active_data.MAU_MP_3.fillna(0.0,inplace = True)

In [98]:
Active_data.MAU_ML_3.value_counts().head(3)

0.0    7862
6.0    1095
1.0    1081
Name: MAU_ML_3, dtype: int64

In [99]:
Active_data.MAU_ML_3.fillna(0.0,inplace = True)

In [100]:
Active_data.MAU_MP_2.value_counts().head(3)

0.0    7664
1.0    4428
2.0    3433
Name: MAU_MP_2, dtype: int64

In [101]:
Active_data.MAU_MP_2.fillna(0.0,inplace = True)

In [102]:
Active_data.MAU_ML_2.value_counts().head(3)

0.0    9290
1.0    1258
3.0    1193
Name: MAU_ML_2, dtype: int64

In [103]:
Active_data.MAU_ML_2.fillna(0.0,inplace = True)

In [104]:
Active_data.MAU_MP_1.fillna(0.0,inplace = True)

In [105]:
Active_data.MAU_ML_1.fillna(0.0,inplace = True)

In [106]:
Active_data.last_login_mp_date_1.value_counts().head(5)

2019-05-31    10935
2019-05-30     4614
2019-05-28     3053
2019-05-29     2549
2019-05-27     2044
Name: last_login_mp_date_1, dtype: int64

In [107]:
priori_ml = Active_data.last_login_ml_date_1.value_counts().head(5)/Active_data.last_login_ml_date_1.value_counts().head(5).sum()

In [108]:
priori_mp = Active_data.last_login_mp_date_1.value_counts().head(5)/Active_data.last_login_mp_date_1.value_counts().head(5).sum()

In [109]:
dates = ['2019-05-31','2019-05-30','2019-05-28','2019-05-29','2019-05-27']
priori_ml_val = priori_ml.values
priori_mp_val = priori_mp.values

In [110]:
Active_data.last_login_mp_date_1.fillna(np.random.choice(dates, replace=True, p= priori_mp_val),inplace = True)

In [111]:
Active_data.last_login_ml_date_1.fillna(np.random.choice(dates, replace=True, p= priori_ml_val),inplace = True)

In [112]:
Active_data.isnull().sum()/Active_data.shape[0]

CUS_CUST_ID_BUY         0.0
MAU_MP_3                0.0
MAU_ML_3                0.0
MAU_MP_2                0.0
MAU_ML_2                0.0
MAU_MP_1                0.0
MAU_ML_1                0.0
last_login_mp_date_1    0.0
last_login_ml_date_1    0.0
dtype: float64

In [113]:
Active_data['last_ml_day'] = pd.DatetimeIndex(Active_data['last_login_ml_date_1']).day
Active_data['last_mp_day'] = pd.DatetimeIndex(Active_data['last_login_mp_date_1']).day

In [114]:
Active_data

,CUS_CUST_ID_BUY,MAU_MP_3,MAU_ML_3,MAU_MP_2,MAU_ML_2,MAU_MP_1,MAU_ML_1,last_login_mp_date_1,last_login_ml_date_1,last_ml_day,last_mp_day
0,424070401,0.0,0.0,1.0,0.0,0.0,0.0,2019-05-31,2019-05-31,31,31
1,271894453,1.0,14.0,11.0,21.0,2.0,12.0,2019-05-17,2019-05-24,24,17
2,327402711,0.0,0.0,0.0,10.0,4.0,26.0,2019-05-28,2019-05-31,31,28
3,310355113,5.0,18.0,5.0,7.0,3.0,10.0,2019-05-21,2019-05-29,29,21
4,59350476,1.0,18.0,5.0,17.0,4.0,27.0,2019-05-31,2019-05-31,31,31
...,...,...,...,...,...,...,...,...,...,...,...
43497,84534152,6.0,28.0,21.0,29.0,20.0,31.0,2019-05-29,2019-05-31,31,29
43498,59319045,12.0,28.0,6.0,24.0,8.0,30.0,2019-05-24,2019-05-31,31,24
43499,80894929,0.0,0.0,0.0,0.0,7.0,0.0,2019-05-31,2019-05-31,31,31
43500,237046723,15.0,21.0,12.0,18.0,18.0,26.0,2019-05-31,2019-05-31,31,31


In [115]:
Final_Data = Final_Data.merge(Active_data,on = 'CUS_CUST_ID_BUY')

* **DINERO_CUENTA.csv:** Contiene datos de la cantidad de dinero en la cuenta y si invierte en fondo.
    * PLATA_CUENTA_1: Máximo dinero en cuenta en el último mes (mes -1).
    * PLATA_CUENTA_2: Máximo dinero en cuenta en el mes -2.
    * INVERSION: Indica en que estado esta el usuario para la inversión:
        * Investing: Está invirtiendo.
        * Elegible: Está disponible para que inviertan dinero.
        * Warmup: Hicieron el onboarding de inversión pero no llenaron los campos.
        * Pending: Hicieron el proceso para invertir, pero está en pending de aprobación.

In [ ]:
#rellena nulls y adentro 

In [116]:
Account_data.isnull().sum()/Account_data.shape[0]

CUS_CUST_ID_BUY    0.000000
PLATA_CUENTA_1     0.000000
PLATA_CUENTA_2     0.044707
INVERSION          0.023807
dtype: float64

In [117]:
Account_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41627 entries, 0 to 41626
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CUS_CUST_ID_BUY  41627 non-null  float64
 1   PLATA_CUENTA_1   41627 non-null  float64
 2   PLATA_CUENTA_2   39766 non-null  float64
 3   INVERSION        40636 non-null  object 
dtypes: float64(3), object(1)
memory usage: 1.3+ MB


In [118]:
len(Account_data.CUS_CUST_ID_BUY.unique())

41627

In [119]:
Account_data.head(5)

,CUS_CUST_ID_BUY,PLATA_CUENTA_1,PLATA_CUENTA_2,INVERSION
0,54018.0,250.99,0.00,pending
1,23513.0,20.05,2220.05,warmup
2,21230.0,0.00,0.00,warmup
3,100387.0,0.00,0.00,eligible
4,97357.0,0.00,0.00,eligible


In [120]:
Account_data.PLATA_CUENTA_2.isnull().sum()

1861

In [121]:
Account_data.PLATA_CUENTA_2.mean()

2741.5737557209677

In [122]:
Account_data.PLATA_CUENTA_2.std()

22387.860360344337

In [123]:
Account_data.PLATA_CUENTA_2.median()

0.0

In [124]:
Account_data.PLATA_CUENTA_2.fillna(0,inplace = True)

In [125]:
Account_data.INVERSION.value_counts()

eligible           18101
warmup              9933
pending             5611
investing           5591
non_eligible         964
disabled             236
disable_pending      117
rejected              53
banned                30
Name: INVERSION, dtype: int64

In [126]:
Account_data.INVERSION.fillna('eligible',inplace = True)

In [127]:
Account_data.head(5)

,CUS_CUST_ID_BUY,PLATA_CUENTA_1,PLATA_CUENTA_2,INVERSION
0,54018.0,250.99,0.00,pending
1,23513.0,20.05,2220.05,warmup
2,21230.0,0.00,0.00,warmup
3,100387.0,0.00,0.00,eligible
4,97357.0,0.00,0.00,eligible


In [128]:
Final_Data = Final_Data.merge(Account_data,on = 'CUS_CUST_ID_BUY', how = 'outer')

In [129]:
Final_Data

,CUS_CUST_ID_BUY,GENDER,TARJETAS,ESTADO,CITY,Rango_edad,1_trans,3_trans,4_trans,6_trans,...,MAU_ML_2,MAU_MP_1,MAU_ML_1,last_login_mp_date_1,last_login_ml_date_1,last_ml_day,last_mp_day,PLATA_CUENTA_1,PLATA_CUENTA_2,INVERSION
0,47629682,male,Credit Card,capital federal,caba,26-30,0,0,2,1,...,7.0,1.0,5.0,2019-05-09,2019-05-31,31,9,0.00,0.00,eligible
1,184288617,male,Account Money,buenos aires,belen de escobar,26-30,5,3,1,0,...,13.0,8.0,19.0,2019-05-30,2019-05-31,31,30,164.29,364.29,pending
2,134359455,female,Debit Card,catamarca,san fernando del valle de catamarca,26-30,0,3,0,0,...,0.0,9.0,5.0,2019-05-28,2019-05-31,31,28,1128.53,1510.54,investing
3,126256149,female,Account Money,buenos aires,lomas de zamora,56-65,0,0,0,1,...,0.0,11.0,0.0,2019-05-29,2019-05-29,29,29,87.77,4.59,warmup
4,447582139,male,Debit Card,rio negro,villa manzano,18-25,0,0,0,2,...,0.0,0.0,0.0,2019-05-31,2019-05-31,31,31,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43497,448290283,Unknown,Unknown,Unknown,Unknown,Unknown,0,0,0,1,...,0.0,0.0,0.0,2019-05-31,2019-05-31,31,31,NaN,NaN,NaN
43498,448313152,Unknown,Unknown,Unknown,Unknown,Unknown,0,0,0,1,...,0.0,0.0,0.0,2019-05-31,2019-05-31,31,31,NaN,NaN,NaN
43499,448326912,Unknown,Unknown,Unknown,Unknown,Unknown,0,0,0,1,...,0.0,0.0,0.0,2019-05-31,2019-05-31,31,31,NaN,NaN,NaN
43500,448330186,Unknown,Unknown,Unknown,Unknown,Unknown,0,0,0,1,...,0.0,0.0,0.0,2019-05-31,2019-05-31,31,31,NaN,NaN,NaN


In [130]:
Final_Data.PLATA_CUENTA_1.fillna(0,inplace = True)

In [131]:
Final_Data.PLATA_CUENTA_2.fillna(0,inplace = True)

In [132]:
Final_Data.INVERSION.fillna('eligible',inplace = True)

In [3]:
(14000 * 100)/43502

32.182428394096824

# MARKETPLACE_DATA:
    * SPENT_ML: Dinero gastado en ML.
    * RECENCY_ML: Fecha de la ultimo compra en ML.
    * FREQUENCY_ML: Cantidad de días que realizó compras.

In [ ]:
#rellena nulls y adentro

In [133]:
MarketPlace_data.head()

,CUS_CUST_ID_BUY,SPENT_ML,RECENCY_ML,FREQUENCY_ML
0,53621056,71.51,2019-03-30,1
1,308208758,269.20,2019-05-30,11
2,30591970,149.22,2019-03-24,4
3,343783095,352.15,2019-05-31,11
4,163876883,1.10,2019-05-10,1


In [134]:
MarketPlace_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24442 entries, 0 to 24441
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CUS_CUST_ID_BUY  24442 non-null  int64  
 1   SPENT_ML         24442 non-null  float64
 2   RECENCY_ML       24442 non-null  object 
 3   FREQUENCY_ML     24442 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 763.9+ KB


In [135]:
MarketPlace_data.CUS_CUST_ID_BUY.unique

<bound method Series.unique of 0         53621056
1        308208758
2         30591970
3        343783095
4        163876883
           ...    
24437    176427668
24438     48969913
24439     97089616
24440     87038369
24441    186829438
Name: CUS_CUST_ID_BUY, Length: 24442, dtype: int64>

In [136]:
MarketPlace_data.RECENCY_ML.value_counts()

2019-05-15    742
2019-05-31    699
2019-05-13    672
2019-05-30    663
2019-05-14    654
             ... 
2018-12-31     16
2018-12-23     15
2019-01-01     13
2018-12-30     12
2018-12-25      6
Name: RECENCY_ML, Length: 183, dtype: int64

In [137]:
MarketPlace_data['RECENCY_ML_DAY'] = pd.DatetimeIndex(MarketPlace_data['RECENCY_ML']).day
MarketPlace_data['RECENCY_ML_MONTH'] = pd.DatetimeIndex(MarketPlace_data['RECENCY_ML']).month
MarketPlace_data['RECENCY_ML_YEAR'] = pd.DatetimeIndex(MarketPlace_data['RECENCY_ML']).year

In [138]:
MarketPlace_data

,CUS_CUST_ID_BUY,SPENT_ML,RECENCY_ML,FREQUENCY_ML,RECENCY_ML_DAY,RECENCY_ML_MONTH,RECENCY_ML_YEAR
0,53621056,71.51,2019-03-30,1,30,3,2019
1,308208758,269.20,2019-05-30,11,30,5,2019
2,30591970,149.22,2019-03-24,4,24,3,2019
3,343783095,352.15,2019-05-31,11,31,5,2019
4,163876883,1.10,2019-05-10,1,10,5,2019
...,...,...,...,...,...,...,...
24437,176427668,13.28,2019-05-09,1,9,5,2019
24438,48969913,1107.82,2019-04-24,9,24,4,2019
24439,97089616,250.63,2019-05-16,3,16,5,2019
24440,87038369,110.39,2019-04-10,2,10,4,2019


In [139]:
Final_Data = Final_Data.merge(MarketPlace_data,on = 'CUS_CUST_ID_BUY', how = 'outer')

In [140]:
Final_Data.SPENT_ML.fillna(0,inplace = True)

In [141]:
Final_Data.RECENCY_ML.fillna('never',inplace = True)

In [142]:
Final_Data

,CUS_CUST_ID_BUY,GENDER,TARJETAS,ESTADO,CITY,Rango_edad,1_trans,3_trans,4_trans,6_trans,...,last_mp_day,PLATA_CUENTA_1,PLATA_CUENTA_2,INVERSION,SPENT_ML,RECENCY_ML,FREQUENCY_ML,RECENCY_ML_DAY,RECENCY_ML_MONTH,RECENCY_ML_YEAR
0,47629682,male,Credit Card,capital federal,caba,26-30,0,0,2,1,...,9,0.00,0.00,eligible,165.98,2018-12-28,2.0,28.0,12.0,2018.0
1,184288617,male,Account Money,buenos aires,belen de escobar,26-30,5,3,1,0,...,30,164.29,364.29,pending,4.70,2019-03-27,1.0,27.0,3.0,2019.0
2,134359455,female,Debit Card,catamarca,san fernando del valle de catamarca,26-30,0,3,0,0,...,28,1128.53,1510.54,investing,401.04,2019-05-26,11.0,26.0,5.0,2019.0
3,126256149,female,Account Money,buenos aires,lomas de zamora,56-65,0,0,0,1,...,29,87.77,4.59,warmup,0.00,never,NaN,NaN,NaN,NaN
4,447582139,male,Debit Card,rio negro,villa manzano,18-25,0,0,0,2,...,31,0.00,0.00,eligible,0.00,never,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43497,448290283,Unknown,Unknown,Unknown,Unknown,Unknown,0,0,0,1,...,31,0.00,0.00,eligible,0.00,never,NaN,NaN,NaN,NaN
43498,448313152,Unknown,Unknown,Unknown,Unknown,Unknown,0,0,0,1,...,31,0.00,0.00,eligible,0.00,never,NaN,NaN,NaN,NaN
43499,448326912,Unknown,Unknown,Unknown,Unknown,Unknown,0,0,0,1,...,31,0.00,0.00,eligible,0.00,never,NaN,NaN,NaN,NaN
43500,448330186,Unknown,Unknown,Unknown,Unknown,Unknown,0,0,0,1,...,31,0.00,0.00,eligible,0.00,never,NaN,NaN,NaN,NaN


In [143]:
Final_Data.fillna(0,inplace = True)

In [144]:
Final_Data.head(5)

,CUS_CUST_ID_BUY,GENDER,TARJETAS,ESTADO,CITY,Rango_edad,1_trans,3_trans,4_trans,6_trans,...,last_mp_day,PLATA_CUENTA_1,PLATA_CUENTA_2,INVERSION,SPENT_ML,RECENCY_ML,FREQUENCY_ML,RECENCY_ML_DAY,RECENCY_ML_MONTH,RECENCY_ML_YEAR
0,47629682,male,Credit Card,capital federal,caba,26-30,0,0,2,1,...,9,0.00,0.00,eligible,165.98,2018-12-28,2.0,28.0,12.0,2018.0
1,184288617,male,Account Money,buenos aires,belen de escobar,26-30,5,3,1,0,...,30,164.29,364.29,pending,4.70,2019-03-27,1.0,27.0,3.0,2019.0
2,134359455,female,Debit Card,catamarca,san fernando del valle de catamarca,26-30,0,3,0,0,...,28,1128.53,1510.54,investing,401.04,2019-05-26,11.0,26.0,5.0,2019.0
3,126256149,female,Account Money,buenos aires,lomas de zamora,56-65,0,0,0,1,...,29,87.77,4.59,warmup,0.00,never,0.0,0.0,0.0,0.0
4,447582139,male,Debit Card,rio negro,villa manzano,18-25,0,0,0,2,...,31,0.00,0.00,eligible,0.00,never,0.0,0.0,0.0,0.0


In [145]:
Final_Data.Rango_edad.value_counts()/Final_Data.Rango_edad.value_counts().sum()

Unknown    0.299664
41-55      0.218886
31-35      0.118500
26-30      0.116592
36-40      0.099559
18-25      0.097605
56-65      0.035516
0-18       0.013678
Name: Rango_edad, dtype: float64

In [146]:
Final_Data = Final_Data.merge(Evaluate_data,on = 'CUS_CUST_ID_BUY')

In [148]:
Final_Data.to_csv('DataVis.csv')

In [151]:
for x in ['TotalSpent', 'PLATA_CUENTA_1', 'PLATA_CUENTA_2', 'SPENT_ML' ,'MAU_MP_3', 'MAU_ML_3', 'MAU_MP_2','MAU_ML_2', 'MAU_MP_1', 'MAU_ML_1']:
    Final_Data[x + 'log'] = np.log(Final_Data[x] + 1)

In [152]:
for x in ['TotalSpent', 'PLATA_CUENTA_1', 'PLATA_CUENTA_2', 'SPENT_ML' ,'MAU_MP_3', 'MAU_ML_3', 'MAU_MP_2','MAU_ML_2', 'MAU_MP_1', 'MAU_ML_1']:
        Final_Data.drop(columns = [x],inplace = True)

In [155]:
Final_Data.head(3)

,CUS_CUST_ID_BUY,GENDER,TARJETAS,ESTADO,CITY,Rango_edad,1_trans,3_trans,4_trans,6_trans,...,TotalSpentlog,PLATA_CUENTA_1log,PLATA_CUENTA_2log,SPENT_MLlog,MAU_MP_3log,MAU_ML_3log,MAU_MP_2log,MAU_ML_2log,MAU_MP_1log,MAU_ML_1log
0,47629682,male,Credit Card,capital federal,caba,26-30,0,0,2,1,...,3.546036,0.000000,0.000000,5.117874,0.000000,0.693147,1.098612,2.079442,0.693147,1.791759
1,184288617,male,Account Money,buenos aires,belen de escobar,26-30,5,3,1,0,...,5.745769,5.107702,5.900692,1.740466,2.197225,3.178054,2.302585,2.639057,2.197225,2.995732
2,134359455,female,Debit Card,catamarca,san fernando del valle de catamarca,26-30,0,3,0,0,...,4.729263,7.029557,7.320884,5.996552,1.386294,0.000000,2.302585,0.000000,2.302585,1.791759


In [162]:
Final_Data.drop(columns = ['last_login_mp_date_1','last_login_ml_date_1','RECENCY_ML'],inplace = True)

In [163]:
Final_Data.columns

Index(['CUS_CUST_ID_BUY', 'GENDER', 'TARJETAS', 'ESTADO', 'CITY', 'Rango_edad',
       '1_trans', '3_trans', '4_trans', '6_trans', '2_trans', '5_trans',
       '7_trans', 'total_transactions', 'Discount_Mean', 'sell_id_251693291',
       'sell_id_254303242', 'sell_id_295153733', 'sell_id_226295793',
       'Cellphone Recharge', 'Transport', 'Instore', 'Utilities',
       'Money Transfer', 'last_ml_day', 'last_mp_day', 'INVERSION',
       'FREQUENCY_ML', 'RECENCY_ML_DAY', 'RECENCY_ML_MONTH', 'RECENCY_ML_YEAR',
       'churn', 'TotalSpentlog', 'PLATA_CUENTA_1log', 'PLATA_CUENTA_2log',
       'SPENT_MLlog', 'MAU_MP_3log', 'MAU_ML_3log', 'MAU_MP_2log',
       'MAU_ML_2log', 'MAU_MP_1log', 'MAU_ML_1log'],
      dtype='object')

In [172]:
cat_vars = ['GENDER', 'TARJETAS', 'ESTADO', 'CITY', 'Rango_edad','last_ml_day', 'last_mp_day', 'INVERSION','RECENCY_ML_DAY','RECENCY_ML_MONTH', 'RECENCY_ML_YEAR']

In [173]:
contin_vars = ['1_trans', '3_trans', '4_trans', '6_trans', '2_trans', '5_trans',
       '7_trans','total_transactions', 'Discount_Mean', 'sell_id_251693291',
       'sell_id_254303242', 'sell_id_295153733', 'sell_id_226295793',
       'Cellphone Recharge', 'Transport', 'Instore', 'Utilities',
       'Money Transfer','FREQUENCY_ML','TotalSpentlog', 'PLATA_CUENTA_1log', 'PLATA_CUENTA_2log',
       'SPENT_MLlog', 'MAU_MP_3log', 'MAU_ML_3log', 'MAU_MP_2log',
       'MAU_ML_2log', 'MAU_MP_1log', 'MAU_ML_1log']

In [167]:
len(cat_vars)

11

In [168]:
len(contin_vars)

29

In [169]:
29 + 11

40

In [171]:
len(Final_Data.columns)

42

In [175]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder,StandardScaler

cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]   

mapper_cat = DataFrameMapper(cat_maps)
Final_Data[cat_vars] = mapper_cat.fit_transform(Final_Data)

mapper_cont = DataFrameMapper(contin_maps)
Final_Data[contin_vars] = mapper_cont.fit_transform(Final_Data)

In [177]:
Final_Data.to_csv('Data_Model.csv')